In [1]:
# Importing the required Libraries
import pandas as pd
import urllib.request
import os

# Reading Excel files
shoe_data = pd.read_excel("Internship_data.xlsx")

# Generalising the program by retrieving the 'view' columns from the dataset using '.iloc'
views = shoe_data.iloc[:,1:6]
views

# Saving the unique 'class' values in a variable
labels = shoe_data['class'].unique()
labels

# Using 'makedirs' function from 'os' library to create  6 subdirectories bearing the class name. Also using the
# 'try-except' method for confirmation/failure of directory creation
for sub_dirs in labels:
    path = '/home/dru_6/Pictures/shoes/' + sub_dirs
    try:
        os.makedirs(path)
    except OSerror:
        print("Creation of the directory %s failed" % path)
    else:  
        print ("Successfully created the directory %s" % path)
        
# Downloading the images from the URLs using 'for' loop
count = 0
path = '/home/dru_6/Pictures/shoes/'
for sub_dirs in labels:
    for images in views:
        for urls in (shoe_data[images][shoe_data['class']==sub_dirs]):
            URL = str(urls)
            #print(URL)
            if (URL != 'nan'):
                urllib.request.urlretrieve(URL, path + sub_dirs + '/' +str(count) + '.jpg')
                count = count + 1

# Importing the 'Keras' library
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.models import Model
from keras import optimizers

# Generating the training data
train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False)

# Generating the validation data
valid_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False)

train_generator = train_datagen.flow_from_directory(
    directory="/content/gdrive/My Drive/train/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

valid_generator = valid_datagen.flow_from_directory(
    directory=r"/content/gdrive/My Drive/valid",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

# Optimizing and training the model before performing data augmentation 
model =  VGG_16()
opt = optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, decay=1e-5/5, amsgrad=False, clipnorm = 1.)
model.compile(loss='categorical_crossentropy', optimizer=opt,
              metrics=['accuracy'])

def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

model.fit_generator(
                train_generator,
                steps_per_epoch=len(train_generator),
                epochs=30,
                verbose=1,
                #callbacks= callbacks, #
                validation_data=valid_generator,
                validation_steps=len(valid_generator))

# Removing the background noise

# Reading the image using cv2
import cv2
pic = cv2.imread('/content/gdrive/My Drive/train/backstrap/0.jpg',1)

## (1) Convert to gray, and threshold
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

## (2) Morph-op to remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

## (3) Find the max-area contour
cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
cnt = sorted(cnts, key=cv2.contourArea)[-1]

## (4) Crop and save it
x,y,w,h = cv2.boundingRect(cnt)
dst = img[y:y+h, x:x+w]
cv2.imwrite("001.png", dst)

# Splitting the data into training and validation set
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    directory="/content/gdrive/My Drive/shoes_1/",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    directory="/content/gdrive/My Drive/shoes_1/",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb',
    shuffle=True,
    subset='validation') # set as validation data

# Optimizing and training the model after performing data augmentation 
model =  VGG_16()
opt = optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, decay=1e-5/5, amsgrad=False, clipnorm = 1.)
model.compile(loss='categorical_crossentropy', optimizer=opt,
              metrics=['accuracy'])

def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),
    validation_data = validation_generator, 
    validation_steps = len(validation_generator),
    epochs = 30,
    verbose = 1)

# Further training the model with incereased number of epochs to increase the accuracy and reduce the loss
model.fit_generator(
    train_generator,
    steps_per_epoch = len(train_generator),
    validation_data = validation_generator, 
    validation_steps = len(validation_generator),
    epochs = 50,
    verbose = 1)

NameError: name 'OSerror' is not defined